In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import keras 
from keras.models import Sequential 
from keras.layers import Dense
from keras.optimizers import Adam 
from keras.utils.np_utils import to_categorical 
from keras.layers import Dropout ,Flatten 
from keras.layers.convolutional import Conv2D , MaxPooling2D
import pickle
import pandas as pd 
import random
import cv2

In [ ]:
np.random.seed(0)


In [ ]:
#download the dataset and the labels and prepare the workflow
!wget https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/traffic-signs-data.zip
!mkdir german-traffic-signs
!unzip traffic-signs-data.zip -d german-traffic-signs
!rm traffic-signs-data.zip
!git clone https://github.com/adelbennaceur/traffic-signs-recognition
!mv traffic-signs-recognition/data/signnames.csv ./german-traffic-signs


In [ ]:
with open('german-traffic-signs/train.p','rb') as f :
  train_data = pickle.load(f)

with open('german-traffic-signs/valid.p','rb') as f :
  val_data = pickle.load(f)
  
with open('german-traffic-signs/test.p','rb') as f :
  test_data = pickle.load(f)
  
print(type(train_data))
     
X_train , y_train = train_data['features'] , train_data['labels']
X_val , y_val  = val_data['features'] , val_data['labels']
X_test , y_test = test_data['features'] , test_data['labels']


In [ ]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

In [ ]:
assert(X_train.shape[0] == y_train.shape[0]) , "Error the number of images does not equal the number of labels"
assert(X_test.shape[0] ==y_test.shape[0]) , "Error the number of images does not equal the number of labels"
assert(X_test.shape[0] == y_test.shape[0]) , "Error the number of images does not equal the number of labels"

assert(X_train.shape[1:] == (32,32,3)) , "the dimension of images are not 32x32x3"
assert(X_val.shape[1:] == (32,32,3)) , "the dimension of images are not 32x32x3"
assert(X_test.shape[1:] == (32,32,3)) , "the dimension of images are not 32x32x3"

In [ ]:
data = pd.read_csv('german-traffic-signs/signnames.csv')
print(data)
num_of_samples = []

cols = 5
num_classes = 43

fig, axs = plt.subplots(nrows=num_classes, ncols = cols, figsize=(5, 50))
fig.tight_layout()
for i in range(cols) :
    for j , row in data.iterrows():
        x_selected = X_train[y_train == j]
        axs[j][i].imshow(x_selected[random.randint(0, len(x_selected - 1)), :, :], cmap=plt.get_cmap("gray"))
        axs[j][i].axis("off")
        if i == 2:
            axs[j][i].set_title(str(j) + "_" +row['SignName'])
            num_of_samples.append(len(x_selected))


In [ ]:
plt.figure(figsize=(12, 4))
plt.bar(range(0, num_classes), num_of_samples)
plt.title("Distribution of the training dataset")
plt.xlabel("Class number")
plt.ylabel("# of images")

In [ ]:
import cv2 
plt.imshow(X_train[1000])
plt.axis('off')
print(X_train[1000].shape)
print(y_train[1000])

In [ ]:
#convert to grayscale 
def grayscale(img):
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  return img 
print(X_train[1000].shape)
img = grayscale(X_train[1000])

plt.imshow(img)
plt.axis('off')
print(img.shape)

In [ ]:
def equalize_hist(img):
 img = cv2.equalizeHist(img)
 return img
img = equalize_hist(img)
plt.imshow(img)
plt.axis('off')


In [ ]:
def preprocessing(img):
  img = grayscale(img)
  img = equalize_hist(img)
  img= img/255
  return img 
X_train  = np.array(list(map(preprocessing,X_train)))
X_val  = np.array(list(map(preprocessing,X_val)))
X_test  = np.array(list(map(preprocessing,X_test)))


In [ ]:
plt.imshow(X_train[random.randint(0,len(X_train)-1)])
plt.axis('off')
print(X_train.shape)

In [ ]:
X_train = X_train.reshape(34799, 32, 32 , 1)
X_val = X_val.reshape(4410, 32, 32, 1)
X_test = X_test.reshape(12630, 32, 32, 1)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(width_shift_range=0.1,
                             height_shift_range=0.1,
                             zoom_range = 0.2,
                             shear_range= 0.15,
                             rotation_range = 30) 
datagen.fit(X_train)

In [ ]:
batches = datagen.flow(X_train,y_train,batch_size = 20)
X_batch , y_batch = next(batches)
fig , axs = plt.subplots(1,15,figsize=(20,5))
fig.tight_layout()
for i in range(15):
  axs[i].imshow(X_batch[i].reshape(32,32))
  axs[i].axis('off')

In [ ]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

In [ ]:
y_train  = to_categorical(y_train, 43)
y_val  = to_categorical(y_val, 43)
y_test  = to_categorical(y_test, 43)
print(y_train.shape)

In [ ]:
def model():
  model  = Sequential()
  
  model.add(Conv2D(60,(5,5),input_shape = (32,32,1) , activation = 'relu'))
  model.add(Conv2D(60,(5,5), activation = 'relu'))
  model.add(MaxPooling2D(pool_size = (2,2)))#(2,2) i.e 32x32x1 ---> 16x16x1
  
  model.add(Conv2D(30,(3,3),activation = 'relu'))
  model.add(Conv2D(30,(3,3),activation = 'relu'))
  model.add(MaxPooling2D(pool_size = (2,2)))  
  model.add(Dropout(0.5))

  model.add(Flatten())
  model.add(Dense(500,activation = 'relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes,activation = 'softmax'))
  model.compile(Adam(lr=0.00099), loss="categorical_crossentropy", metrics=["accuracy"])
  return model

In [ ]:
model = model()
print(model.summary())

In [ ]:
print('Starting training.....')
history  = model.fit_generator(datagen.flow(X_train , y_train , batch_size = 50), steps_per_epoch = 2000,epochs = 10,validation_data=(X_val,y_val),shuffle = 1 ,verbose = 1 ) 

In [ ]:
#saving the model your drive.

from google.colab import drive 
drive.mount("/content/drive")

In [ ]:
import os 
model_path = "/content/drive/My Drive/saved_model"
if not os.path.exists(model_path):
    os.mkdir(model_path)
model.save(os.path.join(model_path,"traffic_sign_model.h5"))

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['training' , 'validation'])
plt.title('loss')
plt.xlabel('epoch')

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['training' , 'validation'])
plt.title('Accuracy')
plt.xlabel('epoch')

In [ ]:

score =  model.evaluate(X_test,y_test,verbose = 1)
print('Test score: ' , score[0])
print('Test accuracy: ' , score[1])

 

In [ ]:
import requests
import cv2
from PIL import Image
url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSq2yVX_FxsRNF6VFgyxFFqaxpf6aZ3sbjWaONlneQHX768CTSEtA'
r = requests.get(url, stream=True)
img = Image.open(r.raw)
plt.imshow(img, cmap=plt.get_cmap('gray'))

In [ ]:

img = np.asarray(img)
img = cv2.resize(img, (32, 32))
img = preprocessing(img)
plt.imshow(img, cmap = plt.get_cmap('gray'))
print(img.shape)

 

In [ ]:
img = img.reshape(1, 32, 32, 1)

In [ ]:
print("predicted sign: "+ str(model.predict_classes(img)))

In [ ]:
model.save('trafic_sign.h5')